In [1]:
import asyncio
import nest_asyncio
import motor.motor_asyncio
from config import *

nest_asyncio.apply()

In [2]:
client = motor.motor_asyncio.AsyncIOMotorClient(connectionString)
db = client.test

try:
    print(client.server_info())
except Exception:
    print("Unable to connect to the server.")

db = client['masters']
collection = db.books
collection

<Future pending cb=[_chain_future.<locals>._call_check_cancel() at C:\Users\User\AppData\Local\Programs\Python\Python310\lib\asyncio\futures.py:384]>


AsyncIOMotorCollection(Collection(Database(MongoClient(host=['masters-shard-00-02.cpq5u.mongodb.net:27017', 'masters-shard-00-01.cpq5u.mongodb.net:27017', 'masters-shard-00-00.cpq5u.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=False, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-wrhudr-shard-0', ssl=True, driver=DriverInfo(name='Motor', version='2.5.1', platform='asyncio')), 'masters'), 'books'))

In [3]:
async def do_find(dataset):
    result = []
    cursor = db[dataset].find({})
    for document in await cursor.to_list(length=1400):
        result.append(document)
    return result

loop = asyncio.get_event_loop()
rates = loop.run_until_complete(do_find('rates'))
# rates = rates[0:10]


In [4]:
len(rates)

61

In [175]:
books = [a['bookId'] for a in (list({v['bookId']:v for v in rates}.values()))]
print(len(books))
users = [a['userId'] for a in (list({v['userId']:v for v in rates}.values()))]
print(len(users))
# books

10
10


In [205]:
# for user in users:
#     print(user)
#     for rate in rates:
#         if rate['userId'].__str__() == user.__str__():
#             print('\t ' + rate['bookId'].__str__())

In [206]:
import math

def sim_pearson_dict(data, item1, item2):
    item1Rates = [d for d in data if d['bookId'].__str__() == item1.__str__()]
    item2Rates = [d for d in data if d['bookId'].__str__() == item2.__str__()]

    n = 0
    sum1 = 0
    sum2 = 0
    sum1Sq = 0
    sum2Sq = 0
    pSum = 0

    for i in item1Rates:
        for j in item2Rates:
            if i['userId'].__str__() == j['userId'].__str__():
                n += 1
                sum1 += i['rate']
                sum2 += j['rate']
                sum1Sq += pow(i['rate'], 2)
                sum2Sq += pow(j['rate'], 2)
                pSum += i['rate'] * j['rate']

    if n == 0:
        return 0

    num = pSum - (sum1 * sum2 / n)
    den = math.sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    
    if den == 0: 
        return 0

    r = ((num / den) + 1) / 2
    return r

In [207]:
def sim_distance(data, item1, item2):
    item1Rates = [d for d in data if d['bookId'].__str__() == item1.__str__()]
    item2Rates = [d for d in data if d['bookId'].__str__() == item2.__str__()]

    sum = 0

    for i in item1Rates:
        for j in item2Rates:
            if i['userId'].__str__() == j['userId'].__str__():
                sum += pow(i['rate'] - j['rate'], 2)
                
    return 1 / (1 + sum)

In [208]:
def topMatches(similarity = sim_pearson_dict):
    scores = []
    result = {}

    for i in range(len(books)):
        temp = []
        # for j in range(i + 1, len(books)):
        for j in range(len(books)):
            if(i != j):
                temp.append((similarity(rates, books[i], books[j]), books[j]))

        temp.sort()
        temp.reverse()
        scores.append(temp)

        result[books[i]] = temp   

    # print(result)     
    return result

In [209]:
def getRecommendedItems(data, itemMatch, user):
    userRatings = [d for d in data if d['userId'].__str__() == user.__str__()]
    # print(userRatings)
    scores = {}
    totalSim = {}

    for userRating in userRatings:
        for (similarity, item2) in itemMatch[userRating['bookId']]:
            if any(d['bookId'].__str__() == item2.__str__() for d in userRatings):
                continue

            scores.setdefault(item2, 0)
            scores[item2] += similarity * userRating['rate']

            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity

    rankings = [(score / totalSim[item], item) for item, score in scores.items() if totalSim[item] != 0]
    
    rankings.sort()
    rankings.reverse()
    print(rankings)
    # return rankings

In [210]:
scores = topMatches()
# scores
# getRecommendedItems(rates, scores, users[0])
for user in users:
    print(user)
    getRecommendedItems(rates, scores, user)
    # print(getRecommendedItems(rates, scores, user))

620bb9fef23b1bc78052c5e1
[(2.2399689827182243, ObjectId('620ea6721feee707fc54939b')), (1.9014462635863216, ObjectId('620ea6721feee707fc54938a'))]
620bb9fef23b1bc78052c5eb
[(4.0, ObjectId('620ea6721feee707fc54939a')), (3.6666666666666665, ObjectId('620ea6721feee707fc54938d')), (3.157797049135763, ObjectId('620ea6721feee707fc54939b'))]
620bb9fef23b1bc78052c5f1
[(3.6666666666666665, ObjectId('620ea6721feee707fc54938d')), (1.0, ObjectId('620ea6721feee707fc54939a'))]
620bb9fef23b1bc78052c5e7
[(4.096571361754434, ObjectId('620ea6721feee707fc54939b')), (4.0, ObjectId('620ea6721feee707fc54938d')), (3.77462333909484, ObjectId('620ea6721feee707fc54938b')), (1.0, ObjectId('620ea6721feee707fc54939a'))]
620bb9fef23b1bc78052c5dd
[(3.0, ObjectId('620ea6721feee707fc54939a')), (2.3333333333333335, ObjectId('620ea6721feee707fc54938d'))]
620bb9fef23b1bc78052c5fb
[(4.349149920718969, ObjectId('620ea6721feee707fc549387')), (4.276338111038169, ObjectId('620ea6721feee707fc54939b')), (3.427601343070517, Objec

In [154]:
# critics = {'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 
# 'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 
# 'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0},
# 'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0,'You, Me and Dupree': 2.5}, 
# 'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 
# 'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
# 'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

critics = {'Lisa Rose': {'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, }, 
'Mick LaSalle': {'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})

            result[item][person] = prefs[person][item]
    return result

filmCritics = transformPrefs(critics)


In [155]:
import datetime
print(type(critics['Lisa Rose']))

rates = []
films = []
users = []

for i in critics:
    for j in critics[i]:
        temp = {
            'critic': i,
            'film': j,
            'rate': critics[i][j],
            'createdAt': datetime.datetime.utcnow().strftime('%d.%m.%Y %H:%M:%S')
        }
        rates.append(temp)
        if j not in films:
            films.append(j)
        if i not in users:
            users.append(i)

# rates   
    

<class 'dict'>


In [156]:
import math

def sim_pearson_dict(data, item1, item2):
    item1Rates = [d for d in data if d['film'] == item1]
    item2Rates = [d for d in data if d['film'] == item2]

    n = 0
    sum1 = 0
    sum2 = 0
    sum1Sq = 0
    sum2Sq = 0
    pSum = 0

    for i in item1Rates:
        for j in item2Rates:
            if i['critic'] == j['critic']:
                n += 1
                sum1 += i['rate']
                sum2 += j['rate']
                sum1Sq += pow(i['rate'], 2)
                sum2Sq += pow(j['rate'], 2)
                pSum += i['rate'] * j['rate']

    if n == 0:
        return 0

    num = pSum - (sum1 * sum2 / n)
    den = math.sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    
    if den == 0: 
        return 0

    r = num / den
    return r

In [157]:
def sim_distance(data, item1, item2):
    item1Rates = [d for d in data if d['film'] == item1]
    item2Rates = [d for d in data if d['film'] == item2]

    sum = 0

    for i in item1Rates:
        for j in item2Rates:
            if i['critic'] == j['critic']:
                sum += pow(i['rate'] - j['rate'], 2)
                
    return 1 / (1 + sum)

In [158]:
def topMatches(similarity = sim_distance):
    # print(prefs)
    # print(person)
    # scores = [(similarity(prefs, item, other), other['film']) for other in prefs if other['film'] != item]
    # scores = [(similarity(rates, item1, item2), item1, item2) for item1 in films for item2 in films if item1 != item2]

    scores = []
    result = {}

    for i in range(len(films)):
        temp = []
        # for j in range(i + 1, len(films)):
        for j in range(len(films)):
            if(i != j):
                temp.append((similarity(rates, films[i], films[j]), films[j]))

        temp.sort()
        temp.reverse()
        scores.append(temp)

        result[films[i]] = temp        
    return result

    scores.sort()
    scores.reverse()
    return scores

In [159]:
scores = topMatches()
# scores

In [160]:
def getRecommendedItems(data, itemMatch, user):
    userRatings = [d for d in data if d['critic'] == user]
    print(userRatings)
    scores = {}
    totalSim = {}

    # Цикл по образцам, оцененным данным пользователем
    for userRating in userRatings:
        # Цикл по образцам, похожим на данный
        for (similarity, item2) in itemMatch[userRating['film']]:
        # Пропускаем, если пользователь уже оценивал данный образец
            if any(d['film'] == item2 for d in userRatings):
                continue

            # Взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item2, 0)
            scores[item2] += similarity * userRating['rate']

            # Сумма всех коэффициентов подобия
            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity

    # Делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить
    # среднее
    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]
    
    # Возвращает список rankings, отсортированный по убыванию
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [168]:
getRecommendedItems(rates, scores, users[6])

[{'critic': 'Toby', 'film': 'Snakes on a Plane', 'rate': 4.5, 'createdAt': '23.02.2022 11:37:46'}, {'critic': 'Toby', 'film': 'You, Me and Dupree', 'rate': 1.0, 'createdAt': '23.02.2022 11:37:46'}, {'critic': 'Toby', 'film': 'Superman Returns', 'rate': 4.0, 'createdAt': '23.02.2022 11:37:46'}]


[(3.238805970149254, 'The Night Listener'),
 (3.021739130434783, 'Just My Luck'),
 (2.851063829787234, 'Lady in the Water')]

In [168]:
import math

def sim_pearson(prefs, p1, p2):
    si = {}
    for item in prefs[p1]: # всі чуваки, які оцінили фільм
        if item in prefs[p2]: # чи оцінив цей чувак інший фільм
            si[item] = 1

    n = len(si)
    # print(si)
    if n == 0: 
        return 0
    
    sum1 = sum([prefs[p1][it] for it in si])
    sum2 = sum([prefs[p2][it] for it in si])

    sum1Sq = sum([pow(prefs[p1][it], 2) for it in si])
    sum2Sq = sum([pow(prefs[p2][it], 2) for it in si])

    pSum = sum([prefs[p1][it] * prefs[p2][it] for it in si])

    num = pSum - (sum1 * sum2 / n)
    den = math.sqrt((sum1Sq - pow(sum1, 2) / n) * (sum2Sq - pow(sum2, 2) / n))
    if den == 0: 
        return 0

    r = num / den
    return r


In [169]:
sim_pearson(filmCritics, 'Snakes on a Plane', 'You, Me and Dupree')

-0.6454972243679047

In [224]:
def topMatches(prefs, person, n=5, similarity = sim_pearson):
 scores = [(similarity(prefs, person, other), other) for other in prefs if other != person]
 
 scores.sort()
 scores.reverse()
 return scores  #[0:n]

In [225]:
scores=[topMatches(filmCritics, item, similarity = sim_pearson) for item in films]
# scores

In [226]:
def getRecommendations(prefs, person, similarity = sim_pearson):
    totals = {}
    simSums = {}
    for other in prefs:
        if other == person: 
            continue
        sim = similarity(prefs, person, other)

        if sim <= 0:
            continue

        for item in prefs[other]:    
            if item not in prefs[person] or prefs[person][item] == 0:
                totals.setdefault(item, 0)
                totals[item] += prefs[other][item] * sim
                simSums.setdefault(item, 0)
                simSums[item] += sim

    rankings = [(total / simSums[item], item) for item, total in totals.items()]

    rankings.sort()
    rankings.reverse()
    return rankings

In [227]:
getRecommendations(critics, 'Toby')

[(3.3477895267131017, 'The Night Listener'),
 (2.8325499182641614, 'Lady in the Water'),
 (2.530980703765565, 'Just My Luck')]

In [228]:
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})

            result[item][person] = prefs[person][item]
    return result

In [229]:
def sim_distance(prefs,person1,person2):
 # Получить список предметов, оцененных обоими
    si = {}

    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item] = 1

    # Если нет ни одной общей оценки, вернуть 0
    if len(si) == 0: 
        return 0

    # Сложить квадраты разностей
    sum_of_squares = sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in prefs[person1] if item in prefs[person2]])
    return 1 / (1+sum_of_squares)


In [230]:
def calculateSimilarItems(prefs, n = 10):
 # Создать словарь, содержащий для каждого образца те образцы, которые
 # больше всего похожи на него.
    result = {}
    # Обратить матрицу предпочтений, чтобы строки соответствовали образцам
    itemPrefs = transformPrefs(prefs)
    c = 0
    for item in itemPrefs:
    # Обновление состояния для больших наборов данных
        c += 1
        if c % 100 == 0: 
            print("%d / %d" % (c, len(itemPrefs)))
    # Найти образцы, максимально похожие на данный
        scores=topMatches(itemPrefs, item, n = n, similarity = sim_pearson) # sim_pearson
        result[item] = scores        
    return result


In [231]:
itemsim = calculateSimilarItems(critics)
itemsim

{'Lady in the Water': [(0.7637626158259785, 'Snakes on a Plane'),
  (0.4879500364742689, 'Superman Returns'),
  (0.3333333333333333, 'You, Me and Dupree'),
  (-0.6123724356957927, 'The Night Listener'),
  (-0.9449111825230676, 'Just My Luck')],
 'Snakes on a Plane': [(0.7637626158259785, 'Lady in the Water'),
  (0.11180339887498941, 'Superman Returns'),
  (-0.3333333333333333, 'Just My Luck'),
  (-0.5663521139548527, 'The Night Listener'),
  (-0.6454972243679047, 'You, Me and Dupree')],
 'Just My Luck': [(0.5555555555555556, 'The Night Listener'),
  (-0.3333333333333333, 'Snakes on a Plane'),
  (-0.42289003161103106, 'Superman Returns'),
  (-0.4856618642571827, 'You, Me and Dupree'),
  (-0.9449111825230676, 'Lady in the Water')],
 'Superman Returns': [(0.6579516949597695, 'You, Me and Dupree'),
  (0.4879500364742689, 'Lady in the Water'),
  (0.11180339887498941, 'Snakes on a Plane'),
  (-0.1798471947990544, 'The Night Listener'),
  (-0.42289003161103106, 'Just My Luck')],
 'You, Me and

In [232]:
def getRecommendedItems(prefs, itemMatch, user):
    print(prefs)
    userRatings = prefs[user]
    scores = {}
    totalSim = {}

    # Цикл по образцам, оцененным данным пользователем
    for (item, rating) in userRatings.items():
        # Цикл по образцам, похожим на данный
        for (similarity, item2) in itemMatch[item]:
        # Пропускаем, если пользователь уже оценивал данный образец
            if item2 in userRatings: 
                continue

            # Взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item2, 0)
            scores[item2] += similarity * rating

            # Сумма всех коэффициентов подобия
            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity

    # Делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить
    # среднее
    rankings=[(score/totalSim[item],item) for item,score in scores.items( )]
    
    # Возвращает список rankings, отсортированный по убыванию
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [233]:
getRecommendedItems(critics, itemsim, 'Toby')

{'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5, 'The Night Listener': 3.0}, 'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5, 'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 3.5}, 'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0, 'Superman Returns': 3.5, 'The Night Listener': 4.0}, 'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0, 'The Night Listener': 4.5, 'Superman Returns': 4.0, 'You, Me and Dupree': 2.5}, 'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0, 'You, Me and Dupree': 2.0}, 'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0, 'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5}, 'Toby': {'Snakes on a Plane': 4.5, 'You, Me and Dupree': 1.0, 'Superman 

[(3.610031066802183, 'Lady in the Water'),
 (3.5313950341859766, 'The Night Listener'),
 (2.960999860724268, 'Just My Luck')]

In [1]:
books = {'book1': {'1':3.0, '3':1.0, '5': 4.0, '6':2.0, '9':5.0, '10': 2.0},
            'book2': {'2':2.0, '3':5.0, '7':5.0, '8':2.0},
            'book3': {'2':3.0, '4':4.0, '5':4.0, '6':1.0, '9': 3.0, '10': 1.0},
            'book4': {'1':5.0, '3':5.0, '7':3.0, '8': 2.0},
            'book5': {'1':2.0, '3':3.0, '5':1.0, '6':4.0, '9': 3.0, '10':3.0},
            'book6': {'1':2.0, '2':1.0, '4':2.0, '8': 5.0, '10': 4.0},
            'book7': {'2':2.0, '5':2.0, '6':3.0, '7': 3.0, '9':1.0},
            'book8': {'2':3.0, '3':4.0, '4':5.0, '6':1.0, '7':2.0, '8':3.0, '9':4.0},
            'book9': {'1':1.0, '3':5.0, '5':5.0, '8': 1.0, '10': 5.0},
            'book10': {'2':2.0, '4':1.0, '5':4.0, '6':2.0, '7':5.0, '9': 3.0}}

In [2]:
import math

def koef_pearson(prefs, book1, book2):
    if book1 == book2:
        return 1

    si = []
    for item in prefs[book1]: # всі чуваки, які оцінили фільм
        if item in prefs[book2]: # чи оцінив цей чувак інший фільм
            si.append(item)

    n = len(si)
    if n == 0: 
        return 0
    
    avg1 = sum([prefs[book1][it] for it in si]) / n
    avg2 = sum([prefs[book2][it] for it in si]) / n

    num = 0 
    sq_sum1 = 0
    sq_sum2 = 0

    for it in si:
        num += (prefs[book1][it] - avg1) * (prefs[book2][it] - avg2)
        sq_sum1 += pow((prefs[book1][it] - avg1), 2)
        sq_sum2 += pow((prefs[book2][it] - avg2), 2)

    den = math.sqrt(sq_sum1 * sq_sum2)
    if den == 0: 
        return 0

    r = ((num / den) + 1) / 2
    # r = (num / den)
    return round(r, 3)


In [3]:
def topMatches(prefs, person, n = 20):
    scores = [(koef_pearson(prefs, person, other), other) for other in prefs]# if other != person]

    scores.sort()
    scores.reverse()
    return scores  #[0:n]

In [4]:
import pandas as pd

# scores = []

# for book in books:
#     scores.append({other: koef_pearson(books, book, other) for other in books})
#     # print(scores)

# df = pd.DataFrame(scores, index = books.keys())
# df

In [5]:
scores=[topMatches(books, item) for item in books]
# scores

In [6]:
# def getRecommendations(prefs, person):
#     totals = {}
#     simSums = {}
#     for other in prefs:
#         if other == person: 
#             continue
#         sim = koef_pearson(prefs, person, other)

#         if sim <= 0:
#             continue

#         for item in prefs[other]:    
#             if item not in prefs[person] or prefs[person][item] == 0:
#                 totals.setdefault(item, 0)
#                 totals[item] += prefs[other][item] * sim
#                 simSums.setdefault(item, 0)
#                 simSums[item] += sim

#     rankings = [(total / simSums[item], item) for item, total in totals.items()]

#     rankings.sort()
#     rankings.reverse()
#     return rankings

In [7]:
def calculateSimilarItems(prefs, n = 20):
    result = {}
    c = 0
    for item in prefs:
        c += 1
        if c % 100 == 0: 
            print("%d / %d" % (c, len(prefs)))

        scores=topMatches(prefs, item, n = n)
        result[item] = scores        
    return result


In [8]:
def transformPrefs(prefs):
    result = {}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item, {})

            result[item][person] = prefs[person][item]
    return result

In [9]:
rates = transformPrefs(books)

In [22]:
def getRecommendedItems(prefs, itemMatch, user):
    userRatings = prefs[user]
    # print(userRatings)
    # print(itemMatch)
    scores = {}
    totalSim = {}

    # Цикл по образцам, оцененным данным пользователем
    for (item, rating) in userRatings.items():
        # Цикл по образцам, похожим на данный
        for (similarity, item2) in itemMatch[item]:
        # Пропускаем, если пользователь уже оценивал данный образец
            if item2 in userRatings: 
                continue

            # Взвешенная суммы оценок, умноженных на коэффициент подобия
            scores.setdefault(item2, 0)
            scores[item2] += similarity * rating
            print(scores)

            # Сумма всех коэффициентов подобия
            totalSim.setdefault(item2, 0)
            totalSim[item2] += similarity

    # Делим каждую итоговую оценку на взвешенную сумму, чтобы вычислить
    # среднее
    rankings=[(round(score/totalSim[item], 3),item) for item,score in scores.items( )]
    
    # Возвращает список rankings, отсортированный по убыванию
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [23]:
similarity = calculateSimilarItems(books)
recomendations = getRecommendedItems(rates, similarity, '8')
# print(rates['8'])
# data = []
# for item in recomendations:
#     data.append([item[1], item[0]])

# df = pd.DataFrame(data)
# df

recomendations

# similarity

{'book7': 2.0}
{'book7': 2.0, 'book10': 2.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 0.0, 'book3': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 0.0, 'book3': 0.0, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 0.0, 'book3': 0.0, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 0.0, 'book3': 0.0, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 0.0, 'book3': 0.0, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 0.0, 'book3': 0.0, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 0.0, 'book3': 0.0, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 5.0, 'book3': 0.0, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 5.0, 'book3': 0.535, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 5.0, 'book3': 0.535, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 5.0, 'book3': 0.535, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 5.0, 'book3': 0.535, 'book1': 0.0}
{'book7': 2.0, 'book10': 2.0, 'book5': 5.0, 'book3': 

[(3.52, 'book5'),
 (3.196, 'book3'),
 (2.265, 'book1'),
 (2.225, 'book10'),
 (2.027, 'book7')]

In [12]:
scores=[topMatches(books, item) for item in books]
# scores

In [13]:
 # for item in reviews1:
    #     if any(review['userId'] == item['userId'] for review in reviews2):
    #         users.append(item['userId'])

# avg1 = 0
    # avg2 = 0

    # for user in users:
    #     for review1 in reviews1:
    #         if review1['userId'] == user:
    #             avg1 += review1['rate']

    #     for review2 in reviews2:
    #         if review2['userId'] == user:
    #             avg2 += review2['rate']

    # avg1 = avg1 / n
    # avg2 = avg2 / n

In [14]:
dict = [{'1':1, '2': 2}, {'1':1, '2': 2}, {'1':1, '3': 2}]
a = any('1' in item.keys() and '2' in item.keys() for item in dict)
a

True

In [15]:
a = 5
b = 3
c = a if a != 5 else b
c

3

In [18]:
a = {'1' : 2}
a['1']

2